In [51]:
!pip install roboflow

In [52]:
from roboflow import Roboflow
rf = Roboflow(api_key="8OhiqM77Mv6CeutA6yAy")
project = rf.workspace("kunalkewatprojects").project("crop_disease_detection-bl93p")
version = project.version(1)
dataset = version.download("multiclass")

loading Roboflow workspace...
loading Roboflow project...


In [53]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [54]:
# Define data directories
train_dir = '/kaggle/working/Crop_Disease_Detection-1/train'
validation_dir = '/kaggle/working/Crop_Disease_Detection-1/valid'
test_dir = '/kaggle/working/Crop_Disease_Detection-1/test'
print('Done')

Done


In [55]:
# Load CSV files
train_csv = pd.read_csv('/kaggle/input/meta-data-cdd/train_classes.csv')
validation_csv = pd.read_csv('/kaggle/input/meta-data-cdd/validation_classes.csv')
test_csv = pd.read_csv('/kaggle/input/meta-data-cdd/test_classes.csv')
print('Done')

Done


In [56]:
# Define image size
image_size = (224, 224)
batch_size = 256
print('executed')

executed


In [57]:
# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
print('executed')

executed


In [58]:
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [59]:
# Create data generators
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_csv,
    directory=train_dir,
    x_col='filename',  # Replace 'image_filename_column' with the actual column name containing image filenames
    y_col=train_csv.columns[1:],  # Assuming class columns start from index 1, adjust if necessary
    target_size=image_size,
    batch_size=batch_size,
    class_mode='raw'  # Use 'raw' for multilabel classification
)
print('executed')

Found 20832 validated image filenames.
executed


In [60]:
validation_generator = validation_datagen.flow_from_dataframe(
    dataframe=validation_csv,
    directory=validation_dir,
    x_col='filename',
    y_col=validation_csv.columns[1:],
    target_size=image_size,
    batch_size=batch_size,
    class_mode='raw'
)

Found 1877 validated image filenames.


In [61]:
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_csv,
    directory=test_dir,
    x_col='filename',
    y_col=test_csv.columns[1:],
    target_size=image_size,
    batch_size=batch_size,
    class_mode='raw'
)

Found 1168 validated image filenames.


In [62]:
# Define number of classes based on the number of columns representing classes in the CSV file
num_classes = len(train_csv.columns[1:])
print(num_classes)

47


In [63]:
# Build the ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [64]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='sigmoid')(x)  # Use sigmoid activation for multilabel classification

In [65]:
model = Model(inputs=base_model.input, outputs=predictions)

In [66]:
# Freeze layers in the base model
for layer in base_model.layers:
    layer.trainable = True

In [67]:
optimizer = Adam(learning_rate=0.01)

In [68]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [69]:
# Train the model
epochs = 60
history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

Epoch 1/60


Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
W0000 00:00:1711198114.738030      91 hlo_rematerialization.cc:2946] Can't reduce memory use below 11.67GiB (12531337185 bytes) by rematerialization; only reduced to 13.41GiB (14400272864 bytes), down from 18.94GiB (20336238424 bytes) originally
W0000 00:00:1711198125.025059      91 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


82/82 ━━━━━━━━━━━━━━━━━━━━ 343s 3s/step - accuracy: 0.4094 - loss: 0.1150 - val_accuracy: 0.0229 - val_loss: 0.1434
Epoch 2/60
82/82 ━━━━━━━━━━━━━━━━━━━━ 271s 3s/step - accuracy: 0.8258 - loss: 0.0218 - val_accuracy: 0.0229 - val_loss: 0.1384
Epoch 3/60
82/82 ━━━━━━━━━━━━━━━━━━━━ 272s 3s/step - accuracy: 0.8768 - loss: 0.0154 - val_accuracy: 0.0229 - val_loss: 0.1431
Epoch 4/60
82/82 ━━━━━━━━━━━━━━━━━━━━ 275s 3s/step - accuracy: 0.8980 - loss: 0.0128 - val_accuracy: 0.0229 - val_loss: 0.1954
Epoch 5/60
82/82 ━━━━━━━━━━━━━━━━━━━━ 272s 3s/step - accuracy: 0.9255 - loss: 0.0095 - val_accuracy: 0.0224 - val_loss: 0.2001
Epoch 6/60
82/82 ━━━━━━━━━━━━━━━━━━━━ 323s 3s/step - accuracy: 0.9333 - loss: 0.0089 - val_accuracy: 0.0229 - val_loss: 0.3865
Epoch 7/60
82/82 ━━━━━━━━━━━━━━━━━━━━ 324s 3s/step - accuracy: 0.9460 - loss: 0.0068 - val_accuracy: 0.0229 - val_loss: 0.2217
Epoch 8/60
82/82 ━━━━━━━━━━━━━━━━━━━━ 273s 3s/step - accuracy: 0.9589 - loss: 0.0054 - val_accuracy: 0.0053 - val_loss: 0.

In [72]:
# Save the model
model.save('resnet50_model_multilabel_csv.h5')
# Save the model in native Keras format
model.save('resnet50_model_multilabel.keras')

In [75]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test loss: {test_loss}\nTest accuracy: {test_accuracy}')

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 337ms/step - accuracy: 0.8574 - loss: 0.0419
Test loss: 0.04178808256983757
Test accuracy: 0.8544520735740662


In [109]:
model.save('crop_disease_detection_model.keras')

In [115]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test loss: {test_loss}\nTest accuracy: {test_accuracy}')

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 335ms/step - accuracy: 0.8482 - loss: 0.0433
Test loss: 0.04194054380059242
Test accuracy: 0.8544520735740662


In [116]:
import matplotlib.pyplot as plt